In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from shapely.geometry import Point, Polygon, MultiPolygon
import geopandas as gpd
import pandas as pd

In [2]:
usa =gpd.read_file("../data/raw/geo/USA_Census_Populated_Places/USA_Census_Populated_Places.shp")
usa['CityFIPS'] = usa['STFIPS']+usa['PLACEFIPS']
usa.set_index('CityFIPS',inplace=True)

In [3]:
usa_100 = usa.sort_values(by="POP2012", ascending=False).iloc[:100].copy()

In [4]:
usa_100.columns
usa_100 = usa_100[['NAME','ST','POP2012']].copy()

In [17]:
usa_100.to_csv("../data/processed/usa_top100cities.csv")

In [10]:
usa_100.iloc[1]

FID                                                        3685
ObjectID                                                   3567
NAME                                                Los Angeles
CLASS                                                      city
ST                                                           CA
STFIPS                                                       06
PLACEFIPS                                                 44000
HOUSEUNITS                                              1413995
POP2012                                                 3833473
POP_CLASS                                                    10
POP12_SQMI                                              8105.11
SQMI                                                     472.97
geometry      (POLYGON ((-118.247287042436 33.7153170145903,...
Name: 0644000, dtype: object

In [28]:
centroids = {}
for i in range(100):
    place = usa_100.iloc[i].STFIPS + usa_100.iloc[i].PLACEFIPS
    centroids[place] =  [usa_100.iloc[i].geometry.centroid.x,usa_100.iloc[i].geometry.centroid.y]

In [32]:
df_centroids = pd.DataFrame.from_dict(centroids,orient='index',columns = ['x','y'])
df_centroids.index.name = 'city_fips'
df_centroids.to_csv('../data/processed/city_centroids.csv')

In [35]:
health = pd.read_csv('../data/raw/health/500_Cities__Local_Data_for_Better_Health__2018_release.csv',dtype=str)
health_city = health[health.GeographicLevel=='City'].copy()

In [36]:
health_city = health_city[['CityName','StateAbbr','CityFIPS']].copy()
health_city.drop_duplicates(subset='CityFIPS',keep='first',inplace=True)
health_city.set_index('CityFIPS',inplace=True)

In [37]:
health_city

,CityName,StateAbbr
CityFIPS,,
1271000,Tampa,FL
2684000,Warren,MI
4801924,Allen,TX
0667000,San Francisco,CA
0937000,Hartford,CT
0653980,Orange,CA
5502375,Appleton,WI
3912000,Canton,OH
3988000,Youngstown,OH


In [9]:
for fips, row in health_city.iterrows():
    try:
        fname = "{}_{}_{}".format(row['StateAbbr'],row['CityName'],fips)
        city_gdf = usa[usa.index==fips].copy()
        city_gdf2 = gpd.overlay(city_gdf, city_gdf, how='union')
        city_gdf2.to_file(driver='GeoJSON', filename= "../data/processed/city_geo_union/{}.geojson".format(fname))
    except ValueError:
        print(fname)
        pass

Exception ignored in: <bound method Handle.__del__ of <rtree.index.IndexStreamHandle object at 0x11f422358>>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/anaconda3/lib/python3.6/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'
Exception ignored in: <bound method Handle.__del__ of <rtree.index.IndexStreamHandle object at 0x11f422b00>>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/anaconda3/lib/python3.6/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'


HI_Honolulu_15003
